In [1]:
# !pip install elasticsearch==8.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.8 MB/s eta 0:00:00


In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(hosts=["http://elasticsearch:9200"])

In [3]:
# Test the connection
if es.ping():
    print("Successfully connected to Elasticsearch!")
else:
    print("Could not connect to Elasticsearch.")

Successfully connected to Elasticsearch!


In [4]:
es

<Elasticsearch(['http://elasticsearch:9200'])>

In [5]:
es.ping()

True

In [6]:
# Perform a 'match_all' query to fetch all documents from your index (replace 'my_index' with your actual index name)
# response = es.search(index="hwinfo_test", body={
#     "query": {
#         "match_all": {}
#     }
# })

# new query
# response = es.search(index="hwinfo_test", body={
#     "query": {
#         "range": {
#             "Time": {
#                 "gte": "02:06:32.910",
#                 "lte": "02:06:35.000"
#             }
#         }
#     }
# })

response = es.search(index = "hwinfo_test", body = {
    "query": {
        "coreThermalThrottling": {
        "field": "Core_Thermal_Throttling",
        "condition": {"term": {"Core_Thermal_Throttling": 1}}
        }
    }
})

BadRequestError: BadRequestError(400, 'parsing_exception', 'unknown query [coreThermalThrottling]')

In [19]:
# Print the retrieved documents
for hit in response['hits']['hits']:
    print(hit['_source'])

In [8]:
# creating a new index
# index_name = "hwinfo_test" # for testing V1.X
index_name = "hwinfo_test_v2.x" # for testing V2.X
# index_name = "hwinfo" # original

In [9]:
settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Date": {"type": "keyword"},
            "Time": {"type": "keyword"},
            "Core_VIDs_avg_V": {"type": "double"},
            "Core_Clocks_avg_MHz": {"type": "integer"},
            "Ring_LLC_Clock_MHz": {"type": "double"},
            "Core_Usage_avg_percent": {"type": "double"},
            "Core_Temperatures_avg_C": {"type": "double"},
            "Core_Distance_to_TjMAX_avg_C": {"type": "double"},
            "CPU_Package_C": {"type": "integer"},
            "CPU_Package_Power_W": {"type": "double"},
            "PL1_Power_Limit_Static_W": {"type": "double"},
            "PL1_Power_Limit_Dynamic_W": {"type": "double"},
            "PL2_Power_Limit_Static_W": {"type": "double"},
            "PL2_Power_Limit_Dynamic_W": {"type": "double"},
            "Core_Thermal_Throttling": {"type": "double"},
            "CPU_FAN_RPM": {"type": "integer"},
            "GPU_FAN_RPM": {"type": "integer"},
        }
    }
}

# hwinfo_schema = StructType([
#     StructField("Date", StringType(), True),
#     StructField("Time", StringType(), True),
#     StructField("Core_VIDs_avg_V", DoubleType(), True),
#     StructField("Core_Clocks_avg_MHz", IntegerType(), True),
#     StructField("Ring_LLC_Clock_MHz", DoubleType(), True),
#     StructField("Core_Usage_avg_percent", DoubleType(), True),
#     StructField("Core_Temperatures_avg_C", DoubleType(), True),
#     StructField("Core_Distance_to_TjMAX_avg_C", DoubleType(), True),
#     StructField("CPU_Package_C", IntegerType(), True),
#     StructField("CPU_Package_Power_W", DoubleType(), True),
#     StructField("PL1_Power_Limit_Static_W", DoubleType(), True),
#     StructField("PL1_Power_Limit_Dynamic_W", DoubleType(), True),
#     StructField("PL2_Power_Limit_Static_W", DoubleType(), True),
#     StructField("PL2_Power_Limit_Dynamic_W", DoubleType(), True)
# ])

# Create the index
es.indices.create(index=index_name, body=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hwinfo_test_v2.x'})

In [9]:
es.indices.delete(index = index_name, ignore=[404, 400])

/tmp/ipykernel_74/3232768211.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index = index_name, ignore=[404, 400])


ObjectApiResponse({'acknowledged': True})

In [10]:
mapping = es.indices.get_mapping(index="hwinfo_test")
mapping = es.indices.get_mapping(index = "hwinfo_test_v2.x")
print(mapping)

{'hwinfo_test_v2.x': {'mappings': {'properties': {'CPU_FAN_RPM': {'type': 'integer'}, 'CPU_Package_C': {'type': 'integer'}, 'CPU_Package_Power_W': {'type': 'double'}, 'Core_Clocks_avg_MHz': {'type': 'integer'}, 'Core_Distance_to_TjMAX_avg_C': {'type': 'double'}, 'Core_Temperatures_avg_C': {'type': 'double'}, 'Core_Thermal_Throttling': {'type': 'double'}, 'Core_Usage_avg_percent': {'type': 'double'}, 'Core_VIDs_avg_V': {'type': 'double'}, 'Date': {'type': 'keyword'}, 'GPU_FAN_RPM': {'type': 'integer'}, 'PL1_Power_Limit_Dynamic_W': {'type': 'double'}, 'PL1_Power_Limit_Static_W': {'type': 'double'}, 'PL2_Power_Limit_Dynamic_W': {'type': 'double'}, 'PL2_Power_Limit_Static_W': {'type': 'double'}, 'Ring_LLC_Clock_MHz': {'type': 'double'}, 'Time': {'type': 'keyword'}}}}}
